<a href="https://colab.research.google.com/github/MarcelloCeresini/ChessBreaker/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
username = 'MarcelloCeresini'
repository = 'ChessBreaker'

In [3]:
# COLAB ONLY CELLS
try:
    import google.colab
    IN_COLAB = True
    !nvidia-smi             # Check which GPU has been chosen for us
    !rm -rf logs
    #from google.colab import drive
    #drive.mount('/content/drive')
    #%cd /content/drive/MyDrive/GitHub/
    !git clone https://github.com/{username}/{repository}.git
    %cd {repository}
    %ls
except:
    IN_COLAB = False

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Cloning into 'ChessBreaker'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 58 (delta 29), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (58/58), done.
/content/ChessBreaker
config.py  main.ipynb  README.md  utils.py


In [11]:
import numpy as np
import tensorflow as tf
import chess

from utils import plane_dict, Config, x_y_from_position

conf = Config()
board = chess.Board()

legal_moves = board.legal_moves
for move in legal_moves:
    print(move.uci())  
print(legal_moves)


g1h3
g1f3
b1c3
b1a3
h2h3
g2g3
f2f3
e2e3
d2d3
c2c3
b2b3
a2a3
h2h4
g2g4
f2f4
e2e4
d2d4
c2c4
b2b4
a2a4
<LegalMoveGenerator at 0x7f51d3485910 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>


In [30]:
# print(board.fen())
print(list(board.pieces(1,True)))

[8, 9, 10, 11, 12, 13, 14, 15]
1
2
3
4
5


In [22]:
@tf.function
def uniform_tensor(x):
    return tf.cast(tf.fill(conf.BOARD_SHAPE, x), conf.PLANES_DTYPE)

NORMAL_PLANES = tf.zeros((*conf.BOARD_SHAPE, (conf.N_PIECE_TYPES+1)*conf.PAST_TIMESTEPS*2), dtype=conf.PLANES_DTYPE) # adding 1 plane for repetition counter for each timestep for each player

@tf.function
def fill_normal_planes(normal_planes):
    for piece_type in range(conf.N_PIECE_TYPES):
        for color in range(2):
            for position in list(board.pieces(piece_type, color)):
                normal_planes(*x_y_from_position(position), conf.N_PIECE_TYPES*color + piece_type) = 1

@tf.function
def special_input_planes(board):
    return tf.vectorized_map( 
            uniform_tensor, 
            [
                bool(board.Turn),
                board.fullmove_number,
                bool(board.has_kingside_castling_rights(True)),   # True for White
                bool(board.has_queenside_castling_rights(True)),
                bool(board.has_kingside_castling_rights(False)),  # False for Black
                bool(board.has_queenside_castling_rights(False)),
                board.halfmove_clock
            ]
        )
    
def fill_input_planes(board):

    tf.concat([
               fill_normal_planes(tf.roll(NORMAL_PLANES, shift=1, axis=-1)), 
               special_input_planes(board)])

0
4


In [ ]:
!pip install anytree

     |████████████████████████████████| 41 kB 662 kB/s 


In [ ]:
from anytree import Node

def choose_move(board, root_node):
    children = root_node.children
    if root_node.children != []:
        root = np.argmax( [child.value for child in children] ) # choose the move with the highest value to be played
        root.parent = None # To detach the subtree and restart with the next move search

    return board.push(root.name), root

def MTCS(board, root_node):
    legal_moves = board.legal_moves
    if root_node.is_leaf:
        priors = tf.boolean_mask(model(input_planes(board)), utils.mask_moves(list(legal_moves)))
        for move, prior in zip(legal_moves, priors):
            Node(move, parent = root_node, prior = prior)
    for child in 

def complete_search(board):
    root_node = Node("", value=0)
    while not board.is_game_over(claim_draw=True):
        MTCS(board, root_node)
        (new_board, new_root_node) = choose_move(board, root_node)
        board, root_node = new_board, new_root_node

        for move in board.legal_moves:

            Node(move, parent = root)


In [ ]:
help(Node)

Help on class Node in module anytree.node.node:

class Node(anytree.node.nodemixin.NodeMixin, builtins.object)
 |  Node(name, parent=None, children=None, **kwargs)
 |  
 |  Method resolution order:
 |      Node
 |      anytree.node.nodemixin.NodeMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name, parent=None, children=None, **kwargs)
 |      A simple tree node with a `name` and any `kwargs`.
 |      
 |      Args:
 |          name: A name or any other object this node can reference to as idendifier.
 |      
 |      Keyword Args:
 |          parent: Reference to parent node.
 |          children: Iterable with child nodes.
 |          *: Any other given attribute is just stored as object attribute.
 |      
 |      Other than :any:`AnyNode` this class has at least the `name` attribute,
 |      to distinguish between different instances.
 |      
 |      The `parent` attribute refers the parent node:
 |      
 |      >>> from anytree import Node, R

In [ ]:
def training_loop():
    